In [1]:
from mesa import Model
from mesa.time import BaseScheduler
from mesa.space import ContinuousSpace
import pandas as pd
from collections import defaultdict
import networkx as nx

from mesa import Agent
from enum import Enum

In [2]:
def compute_average_driving(model):
    
    return sum(model.drive_times) / len(model.drive_times) if len(model.drive_times) else 0
# ---------------------------------------------------------------
def compute_worst_bridge(model):

    times = {agent.name : agent.total_delay_time / agent.trucks_passed if agent.trucks_passed > 0 else 0 for agent in model.schedule.agents if agent.__class__.__name__ == "Bridge"}
    return max(times, key=times.get)
# ---------------------------------------------------------------
def compute_worst_bridge_delay(model):

    times = {agent.name : agent.total_delay_time / agent.trucks_passed if agent.trucks_passed > 0 else 0 for agent in model.schedule.agents if agent.__class__.__name__ == "Bridge"}
    name = max(times, key=times.get)
    return times[name]
# ---------------------------------------------------------------
def get_probs(model):

    return model.probs



In [3]:
# ---------------------------------------------------------------
def set_lat_lon_bound(lat_min, lat_max, lon_min, lon_max, edge_ratio=0.02):

    lat_edge = (lat_max - lat_min) * edge_ratio
    lon_edge = (lon_max - lon_min) * edge_ratio

    x_max = lon_max + lon_edge
    y_max = lat_min - lat_edge
    x_min = lon_min - lon_edge
    y_min = lat_max + lat_edge
    return y_min, y_max, x_min, x_max


class BangladeshModel(Model):
    """
    The main (top-level) simulation model

    One tick represents one minute; this can be changed
    but the distance calculation need to be adapted accordingly

    Class Attributes:
    -----------------
    step_time: int
        step_time = 1 # 1 step is 1 min

    path_ids_dict: defaultdict
        Key: (origin, destination)
        Value: the shortest path (Infra component IDs) from an origin to a destination

        Only straight paths in the Demo are added into the dict;
        when there is a more complex network layout, the paths need to be managed differently
    

    sources: list
        all sources in the network

    sinks: list
        all sinks in the network
        
    """

    step_time = 1

    file_name = '../data/processed/N1_N2_plus_sideroads.csv'

    def __init__(self, prob_A=0, prob_B=0, prob_C=0, prob_D=0, seed=None, x_max=500, y_max=500, x_min=0, y_min=0):

        self.schedule = BaseScheduler(self)
        self.running = True
        self.path_ids_dict = defaultdict(lambda: pd.Series())
        self.path_ids_dict_complex = defaultdict(list)
        self.space = None
        self.sources = []
        self.sinks = []
        self.probs = {"A": prob_A, "B": prob_B, "C": prob_C, "D": prob_D}
        self.drive_times = []
        self.delay_at_bridge = []

        self.generate_model()

    def generate_model(self):
        """
        generate the simulation model according to the csv file component information

        Warning: the labels are the same as the csv column labels
        """

        df = pd.read_csv(self.file_name)
        
        # constructing network
        G = nx.Graph()

        for i, row in df.iterrows():
            G.add_node(row['id'], pos=[row['lon'],row['lat']]) # adding all nodes from road ids

        # adding edges between all nodes on a road. Intersections have the same id, so will be connected this way too.
        p_row = None
        for i, row in df.iterrows():
            if p_row is not None:
                if p_row['road'] == row['road']:
                    G.add_edge(p_row['id'], row['id'], weight=row['length'])
            p_row = row

        self.network = G
    
        # a list of names of roads to be generated
        roads = df['road'].unique().tolist() 

        df_objects_all = []
        
        for road in roads:
            # Select all the objects on a particular road in the original order as in the cvs
            df_objects_on_road = df[df['road'] == road]

            if not df_objects_on_road.empty:
                df_objects_all.append(df_objects_on_road)

                """
                Set the path 
                1. get the serie of object IDs on a given road in the cvs in the original order
                2. add the (straight) path to the path_ids_dict
                3. put the path in reversed order and reindex
                4. add the path to the path_ids_dict so that the vehicles can drive backwards too
                """
                path_ids = df_objects_on_road['id']
                path_ids.reset_index(inplace=True, drop=True)
                self.path_ids_dict[path_ids[0], path_ids.iloc[-1]] = path_ids
                self.path_ids_dict[path_ids[0], None] = path_ids
                path_ids = path_ids[::-1]
                path_ids.reset_index(inplace=True, drop=True)
                self.path_ids_dict[path_ids[0], path_ids.iloc[-1]] = path_ids
                self.path_ids_dict[path_ids[0], None] = path_ids
        
        # put back to df with selected roads so that min and max and be easily calculated
        df = pd.concat(df_objects_all)
        y_min, y_max, x_min, x_max = set_lat_lon_bound(
            df['lat'].min(),
            df['lat'].max(),
            df['lon'].min(),
            df['lon'].max(),
            0.05
        )

        # ContinuousSpace from the Mesa package;
        # not to be confused with the SimpleContinuousModule visualization
        self.space = ContinuousSpace(x_max, y_max, True, x_min, y_min)

        for df in df_objects_all:
            for _, row in df.iterrows():  # index, row in ...

                # create agents according to model_type
                model_type = row['model_type'].strip()
                agent = None

                name = row['name']
                if pd.isna(name):
                    name = ""
                else:
                    name = name.strip()

                if model_type == 'source':
                    agent = Source(row['id'], self, row['length'], name, row['road'])
                    self.sources.append(agent.unique_id)
                elif model_type == 'sink':
                    agent = Sink(row['id'], self, row['length'], name, row['road'])
                    self.sinks.append(agent.unique_id)
                elif model_type == 'sourcesink':
                    agent = SourceSink(row['id'], self, row['length'], name, row['road'])
                    self.sources.append(agent.unique_id)
                    self.sinks.append(agent.unique_id)
                elif model_type == 'bridge':
                    agent = Bridge(row['id'], self, row['length'], row['bridge_name'], row['road'], row['condition'])
                elif model_type == 'link':
                    agent = Link(row['id'], self, row['length'], name, row['road'])
                elif model_type == 'intersection':
                    if not row['id'] in self.schedule._agents:
                        agent = Intersection(row['id'], self, row['length'], name, row['road'])

                if agent:
                    self.schedule.add(agent)
                    y = row['lat']
                    x = row['lon']
                    self.space.place_agent(agent, (x, y))
                    agent.pos = (x, y)

    def get_random_route(self, source):
        """
        pick up a random route given an origin
        """
        while True:
            # different source and sink
            sink = self.random.choice(self.sinks)
            if sink is not source:
                break
        return self.path_ids_dict[source, sink]

    def get_route(self, source):
        # set random source as destination
        destination = self.random.choice(self.sinks)
        while destination is source:
            destination = self.random.choice(self.sinks)
        # check if path is already known
        if not self.path_ids_dict_complex[source, destination]:
            self.path_ids_dict_complex[source, destination] = nx.shortest_path(self.network, source=source, target=destination, weight='weight')
        # return the path
        return self.path_ids_dict_complex[source, destination]

    def get_straight_route(self, source):
        """
        pick up a straight route given an origin
        """
        return self.path_ids_dict[source, None]

    def step(self):
        """
        Advance the simulation by one step.
        """
        self.schedule.step()



In [5]:
from mesa import Agent
from enum import Enum


# ---------------------------------------------------------------
class Infra(Agent):
    """
    Base class for all infrastructure components

    Attributes
    __________
    vehicle_count : int
        the number of vehicles that are currently in/on (or totally generated/removed by)
        this infrastructure component

    length : float
        the length in meters
    ...

    """

    def __init__(self, unique_id, model, length=0,
                 name='Unknown', road_name='Unknown'):
        super().__init__(unique_id, model)
        self.length = length
        self.name = name
        self.road_name = road_name
        self.vehicle_count = 0

    def step(self):
        pass

    def __str__(self):
        return type(self).__name__ + str(self.unique_id)


# ---------------------------------------------------------------
class Bridge(Infra):
    """
    Creates delay time

    Attributes
    __________
    condition:
        condition of the bridge
    ...
    
    """

    def __init__(self, unique_id, model, length=0,
                 name='Unknown', road_name='Unknown', condition='Unknown'):
        super().__init__(unique_id, model, length, name, road_name)

        self.condition = condition
        self.delay = self.model.probs[condition] > self.random.random()
        self.total_delay_time = 0
        self.trucks_passed = 0

    def get_delay_time(self):
        if self.delay: # depending on the bridge length, return a specific value for each truck arriving at the broken bridge
            if self.length < 10:
                delay_time = self.random.uniform(10, 20)
            elif self.length < 50:
                delay_time = self.random.uniform(15, 60)
            elif self.length < 200:
                delay_time = self.random.uniform(45, 90)
            else:
                delay_time = self.random.triangular(60, 240, 120)
        else:
            delay_time =  0 # no delay if bridge is not broken or under construction
        self.total_delay_time += delay_time
        self.trucks_passed += 1
        return delay_time
# ---------------------------------------------------------------
class Link(Infra):
    pass


# ---------------------------------------------------------------
class Intersection(Infra):
    pass


# ---------------------------------------------------------------
class Sink(Infra):
    """
    Sink removes vehicles

    Attributes
    __________
    vehicle_removed_toggle: bool
        toggles each time when a vehicle is removed
    ...

    """
    vehicle_removed_toggle = False

    def remove(self, vehicle):
        self.model.schedule.remove(vehicle)
        self.model.drive_times.append(vehicle.removed_at_step - vehicle.generated_at_step)
        self.vehicle_removed_toggle = not self.vehicle_removed_toggle
        # print(str(self) + ' REMOVE ' + str(vehicle))
# ---------------------------------------------------------------

class Source(Infra):
    """
    Source generates vehicles

    Class Attributes:
    -----------------
    truck_counter : int
        the number of trucks generated by ALL sources. Used as Truck ID!

    Attributes
    __________
    generation_frequency: int
        the frequency (the number of ticks) by which a truck is generated

    vehicle_generated_flag: bool
        True when a Truck is generated in this tick; False otherwise
    ...

    """

    truck_counter = 0
    generation_frequency = 5
    vehicle_generated_flag = False

    def step(self):
        if self.model.schedule.steps % self.generation_frequency == 0:
            self.generate_truck()
        else:
            self.vehicle_generated_flag = False

    def generate_truck(self):
        """
        Generates a truck, sets its path, increases the global and local counters
        """
        try:
            agent = Vehicle('Truck' + str(Source.truck_counter), self.model, self)
            if agent:
                self.model.schedule.add(agent)
                agent.set_path()
                Source.truck_counter += 1
                self.vehicle_count += 1
                self.vehicle_generated_flag = True
                # print(str(self) + " GENERATE " + str(agent))
        except Exception as e:
            print("Oops!", e.__class__, "occurred.")


# ---------------------------------------------------------------
class SourceSink(Source, Sink):
    """
    Generates and removes trucks
    """
    pass


# ---------------------------------------------------------------
class Vehicle(Agent):
    """

    Attributes
    __________
    speed: float
        speed in meter per minute (m/min)

    step_time: int
        the number of minutes (or seconds) a tick represents
        Used as a base to change unites

    state: Enum (DRIVE | WAIT)
        state of the vehicle

    location: Infra
        reference to the Infra where the vehicle is located

    location_offset: float
        the location offset in meters relative to the starting point of
        the Infra, which has a certain length
        i.e. location_offset < length

    path_ids: Series
        the whole path (origin and destination) where the vehicle shall drive
        It consists the Infras' uniques IDs in a sequential order

    location_index: int
        a pointer to the current Infra in "path_ids" (above)
        i.e. the id of self.location is self.path_ids[self.location_index]

    waiting_time: int
        the time the vehicle needs to wait

    generated_at_step: int
        the timestamp (number of ticks) that the vehicle is generated

    removed_at_step: int
        the timestamp (number of ticks) that the vehicle is removed
    ...

    """

    # 48 km/h translated into meter per min
    speed = 48 * 1000 / 60
    # One tick represents 1 minute
    step_time = 1

    class State(Enum):
        DRIVE = 1
        WAIT = 2

    def __init__(self, unique_id, model, generated_by,
                 location_offset=0, path_ids=None):
        super().__init__(unique_id, model)
        self.generated_by = generated_by
        self.generated_at_step = model.schedule.steps
        self.location = generated_by
        self.location_offset = location_offset
        self.pos = generated_by.pos
        self.path_ids = path_ids
        # default values
        self.state = Vehicle.State.DRIVE
        self.location_index = 0
        self.waiting_time = 0
        self.waited_at = None
        self.removed_at_step = None

    def __str__(self):
        return "Vehicle" + str(self.unique_id) + \
               " +" + str(self.generated_at_step) + " -" + str(self.removed_at_step) + \
               " " + str(self.state) + '(' + str(self.waiting_time) + ') ' + \
               str(self.location) + '(' + str(self.location.vehicle_count) + ') ' + str(self.location_offset)

    def set_path(self):
        """
        Set the origin destination path of the vehicle
        """
        self.path_ids = self.model.get_route(self.generated_by.unique_id)

    def step(self):
        """
        Vehicle waits or drives at each step
        """
        if self.state == Vehicle.State.WAIT:
            self.waiting_time = max(self.waiting_time - 1, 0)
            if self.waiting_time == 0:
                self.waited_at = self.location
                self.state = Vehicle.State.DRIVE

        if self.state == Vehicle.State.DRIVE:
            self.drive()

        """
        To print the vehicle trajectory at each step
        """
        # print(self)

    def drive(self):

        # the distance that vehicle drives in a tick
        # speed is global now: can change to instance object when individual speed is needed
        distance = Vehicle.speed * Vehicle.step_time
        distance_rest = self.location_offset + distance - self.location.length

        if distance_rest > 0:
            # go to the next object
            self.drive_to_next(distance_rest)
        else:
            # remain on the same object
            self.location_offset += distance

    def drive_to_next(self, distance):
        """
        vehicle shall move to the next object with the given distance
        """

        self.location_index += 1
        next_id = self.path_ids[self.location_index]
        next_infra = self.model.schedule._agents[next_id]  # Access to protected member _agents

        if isinstance(next_infra, Sink):
            # arrive at the sink
            self.arrive_at_next(next_infra, 0)
            self.removed_at_step = self.model.schedule.steps
            self.location.remove(self)
            return
        elif isinstance(next_infra, Bridge):
            self.waiting_time = next_infra.get_delay_time()
            if self.waiting_time > 0:
                # arrive at the bridge and wait
                self.arrive_at_next(next_infra, 0)
                self.state = Vehicle.State.WAIT
                return
            # else, continue driving

        if next_infra.length > distance:
            # stay on this object:
            self.arrive_at_next(next_infra, distance)
        else:
            # drive to next object:
            self.drive_to_next(distance - next_infra.length)

    def arrive_at_next(self, next_infra, location_offset):
        """
        Arrive at next_infra with the given location_offset
        """
        self.location.vehicle_count -= 1
        self.location = next_infra
        self.location_offset = location_offset
        self.location.vehicle_count += 1

# EOF -----------------------------------------------------------


In [8]:
from mesa.batchrunner import FixedBatchRunner
import pandas as pd

# run settings
run_length = 5 * 24 * 60 # run the model for 5 * 24 hours, translated to minutes
seed = 1234567
iterations = 10

# parameters that remain constant for all runs
fixed_params = {
    # option for set seed "seed": seed,
}


parameter_list = [
    {"prob_A": 0, "prob_B": 0, "prob_C": 0, "prob_D": 0},
    {"prob_A": 0, "prob_B": 0, "prob_C": 0, "prob_D": 0.05},
    {"prob_A": 0, "prob_B": 0, "prob_C": 0.05, "prob_D": 0.10},
    {"prob_A": 0, "prob_B": 0.05, "prob_C": 0.10, "prob_D": 0.20},
    {"prob_A": 0.05, "prob_B": 0.10, "prob_C": 0.20, "prob_D": 0.40},
]

# configure the batch runner
batch_run = FixedBatchRunner(BangladeshModel, 
                        parameter_list, 
                        fixed_params,
                        iterations=iterations,
                        max_steps=run_length,
                        model_reporters={"Average driving time": compute_average_driving, "Worst bridge name": compute_worst_bridge, "Worst bridge delay": compute_worst_bridge_delay, "Probs": get_probs}
                        )

# run the batch run configuration and save to dataframe
batch_run.run_all()

run_data = batch_run.get_model_vars_dataframe()

# save run data to csv
run_data.to_csv('../data/experiment/all_scenarios.csv')

# save every scenerio to seperate csv file
start = 0
end = iterations 

for scenario in range(len(parameter_list)):
    run_data[start:end].to_csv('../data/experiment/scenario{}.csv'.format(scenario))
    start += iterations
    end += iterations

50it [2:17:36, 165.13s/it]
